## Python + Plotly Express Walkthrough Lab
#### Goal: Learn how to use Python and Plotly Express library to create three types of interactive data visualizations. This lab will focus on learning how to create a bar, line, and scatter plots that tell a clear and insightful story. 

1. Setup and Imports - Start by ensureing the right packages are installed and imported. There is no need to isntall packages as the ones we need for this lab and the assignment were imported during the set up. 

In [ ]:
# import packages
import pandas as pd
import plotly.express as px

2. Example 1 - Bar Chart: Comparing Product Sales
Scenario: You are data analyst tasked with creating a visual to help understand product sale trends in a business unit. You want to compare how each product performed in 2024. 

The first thing we will do is create a dataset manually using the pandas DataFrame constructor. The pandas dataframe is a 2-dimensional labeled data structure with columns and rows. You can think of it like a spreadsheet in excel or SQL table. It is generally the most commonly used pandas object. We will create a simple dataset with product names and their corresponding sales figures. 

In [ ]:
salecs_data = pd.DataFrame({
    "Product": ["A", "B", "C", "D", "E"],
    "Sales": [150, 200, 300, 250, 400]
})

Now that the data has been created we will create a bar chart using the plotly express bar constructor. Here we will invoke pandas express bar constructor to create a bar chart. Note how the x parameter is used to specify the column for the x-axis, the y parameter is used to specify the column for the y-axis, and the title parameter is used to set the title of the chart. Those are the most basic parameters that you'd want to use. In the code below we see other optional paramenters; 
* The text parameter is used to display the sales values on top of each bar
* The color parameter is used to color the bars based on the product names.
Note also how the the resulting bar chart is stored in the fig variable by using the `fig = px...`
Check out this link for detailed documentation of the plotly express bar constructor: https://plotly.github.io/plotly.py-docs/generated/plotly.express.bar.html

In [ ]:
fig = px.bar(
        salecs_data, 
        x="Product", 
        y="Sales", 
        title="Product Sales in 2024",
        text="Sales",
        color="Product"
)